In [1]:
import time

import numpy as np
import matplotlib.pyplot as plt
from typing import List
import pysr
import h5py
import sympy
import numpy as np
from matplotlib import pyplot as plt
from pysr import PySRRegressor
from sklearn.model_selection import train_test_split
from pysr_model import create_model
from plotting import plot_predictions


# Configure plot defaults
plt.rcParams["axes.facecolor"] = "white"
plt.rcParams["grid.color"] = "#666666"

####### Set Input Arguments ########
# This is where you set the args to your function.
# Parameter name
param_name = "tau0"
# take z = 3.6
z = 2.8

# Plotting
quantile_low = 0.16  # quantile for parameter value to fix
quantile_high = 0.84  # quantile for parameter value to fix
####################################

param_dict = {
    "dtau0": 0,
    "tau0": 1,
    "ns": 2,
    "Ap": 3,
    "herei": 4,
    "heref": 5,
    "alphaq": 6,
    "hub": 7,
    "omegamh2": 8,
    "hireionz": 9,
    "bhfeedback": 10,
}
param_idx = param_dict[param_name]  # index of the parameter in the params array

# TODO: Probably also be careful about the filepath~
with h5py.File(
    "../InferenceMultiFidelity/1pvar/lf_{}_npoints50_datacorrFalse.hdf5".format(param_name), "r"
) as file:
    print(file.keys())

    flux_vectors_low = file["flux_vectors"][:]
    kfkms_low = file["kfkms"][:]
    # kfmpc = file["kfmpc"][:]
    zout = file["zout"][:]
    resolution_low=np.full((1750,1),1536)

    params_low = file["params"][:]
#kfkms.shape, flux_vectors.shape, zout.shape, params.shape

with h5py.File(
    "../InferenceMultiFidelity/1pvar/hf_{}_npoints50_datacorrFalse.hdf5".format(param_name), "r"
) as file:
    print(file.keys())

    flux_vectors_hi = file["flux_vectors"][:]
    kfkms_hi = file["kfkms"][:]
    # kfmpc = file["kfmpc"][:]
    zout_hi = file["zout"][:]
    resolution_hi=np.full((1750,1),3072)
    params_hi = file["params"][:]

zindex = np.where(zout == z)[0][0]  # index of z = 5

# take z=3.6, and flatten the flux vectors, such that the dim=1 is p1d values per k and parameter
flux_vectors_z_low = flux_vectors_low[:, zindex, :]
flux_vectors_z_low = flux_vectors_z_low.flatten()[:, np.newaxis]  # add a new axis to make it 2D

flux_vectors_z_hi = flux_vectors_hi[:, zindex, :]
flux_vectors_z_hi = flux_vectors_z_hi.flatten()[:, np.newaxis]  # add a new axis to make it 2D

# do the same for kfkms
kfkms_z_low = kfkms_low[:, zindex, :]
kfkms_z_low = kfkms_z_low.flatten()[:, np.newaxis]  # add a new axis to make it 2D

params_values_low = params_low[:, param_idx]
# repeat this for the number of kfkms
params_values_low = np.repeat(params_values_low[:, np.newaxis], kfkms_low.shape[2], axis=1)
params_values_low = params_values_low.flatten()[:, np.newaxis]  # add a new axis to make it 2D

# Shapes: (1750, 1)
X_param = params_values_low
X_k = kfkms_z_low
y = flux_vectors_z_low

assert(y.shape == (1750, 1))
# Concatenate inputs to form design matrix
X = np.hstack([X_param, X_k])  # shape: (1750, 2)

X_1 = np.hstack([X_param, X_k,resolution_low])  # shape: (1750, 3)
assert(X.shape== (1750, 2))

params_values_hi = params_low[:, param_idx]
# repeat this for the number of kfkms
params_values_hi = np.repeat(params_values_hi[:, np.newaxis], kfkms_low.shape[2], axis=1)
params_values_hi = params_values_hi.flatten()[:, np.newaxis]  # add a new axis to make it 2D

# Shapes: (1750, 1)
X_param_hi = params_values_hi

y_hi = flux_vectors_z_hi

#normalization of y
y_low_mean=np.mean(y, axis=0)
y_low_std=np.std(y, axis=0)
y_low_normalized=(y-y_low_mean)/y_low_std

y_hi_mean=np.mean(y_hi, axis=0)
y_hi_std=np.std(y_hi, axis=0)
y_hi_normalized=(y_hi-y_low_mean)/y_low_std

#stacking
X2=np.hstack([X_param_hi, X_k])
X_2=np.hstack([X_param_hi, X_k,resolution_hi])  # shape: (1750, 3)

#normalization of x
#X_1_normalized=X_1/(np.max(X_1,axis=0)-np.min(X_1,axis=0))
#X_2_normalized=X_2/(np.max(X_2,axis=0)-np.min(X_2,axis=0))
#THROWS ERROR, I BELIEVE BECAUSE OF DIVISION BY 0

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython
<KeysViewHDF5 ['flux_vectors', 'kfkms', 'loo_error', 'params', 'zout']>
<KeysViewHDF5 ['flux_vectors', 'kfkms', 'loo_error', 'params', 'zout']>


In [2]:
X_1_normalized=X_1/(np.max(X_1,axis=0)-np.min(X_1,axis=0))
X_2_normalized=X_2/(np.max(X_2,axis=0)-np.min(X_2,axis=0))


/var/folders/y9/g2jj3nys17lfz6hmth5_mpth0000gn/T/ipykernel_2359/2045620130.py:1: RuntimeWarning: divide by zero encountered in divide
  X_1_normalized=X_1/(np.max(X_1,axis=0)-np.min(X_1,axis=0))
/var/folders/y9/g2jj3nys17lfz6hmth5_mpth0000gn/T/ipykernel_2359/2045620130.py:2: RuntimeWarning: divide by zero encountered in divide
  X_2_normalized=X_2/(np.max(X_2,axis=0)-np.min(X_2,axis=0))


In [4]:
(np.max(X_1,axis=0)-np.min(X_1,axis=0))

array([0.4998  , 0.018428, 0.      ])

In [10]:
with h5py.File(
    "../InferenceMultiFidelity/1pvar/hf_{}_npoints50_datacorrFalse.hdf5".format(param_name), "r"
) as file:
    print(file.keys())

    flux_vectors_hi = file["flux_vectors"][:]
    kfkms_hi = file["kfkms"][:]
    # kfmpc = file["kfmpc"][:]
    zout_hi = file["zout"][:]
    resolution_hi=np.full((1750,1),3072)
    params_hi = file["params"][:]


mean_flux_hi = np.mean(flux_vectors_hi, axis=0)
mean_flux_hi.shape

<KeysViewHDF5 ['flux_vectors', 'kfkms', 'loo_error', 'params', 'zout']>


(13, 35)

In [ ]:
y_low_mean=np.mean(y, axis=0)
